In [1]:
from tickers import Ticker, scrape_tickers, query_list, recent_IPO_list, upcoming_IPO_list

# Scrape to get updated tickers and company names for stocks and ETFs
scrape_tickers()

import_lists = [query_list, recent_IPO_list, upcoming_IPO_list]

tickers = []

# Build raw query list with ticker abbrevation and name of company
for i in import_lists:
    for item in i:
        if item.abbrev == 'A' or item.abbrev == 'I':
            item.abbrev = '-'
        tickers.append(item)

In [2]:
import pandas as pd

submissions_df = pd.read_csv('submissions.csv')
comments_df = pd.read_csv('comments.csv')

In [3]:
submissions_df['Body_Words'] = submissions_df.Body.str.replace("'", "").str.strip().str.split('[\W_]+')
submissions_df['Title_Words'] = submissions_df.Title.str.replace("'", "").str.strip().str.split('[\W_]+')

In [4]:
common_words = ['Corp', 'Corporation', 'Ltd', 'Acquisition', 'Pharmaceuticals', 'Holding', 'Group']
new_name = ""
ticker_names = []
for ticker in tickers:
    new_name = ""
    company_words = ticker.name.split()
    for word in company_words:
        if word in common_words:
            continue
        else:
            if new_name == "":
                new_name = word
            else:
                new_name = new_name + " " + word
    ticker.name = new_name

In [5]:
submissions_df["Tickers"] = ""
for index, row in submissions_df.iterrows():
    tickers_found = []
    body_list = row[12]
    title_list = row[13]
    body_set = set(body_list)
    title_set = set(title_list)
    for ticker in tickers:
        if ticker.abbrev == 'A' or ticker.abbrev == 'I':
            #if (ticker.name.split()[0] in body_set) or (ticker.name.split()[0] in title_set):
            if (ticker.name in body_set) or (ticker.name in title_set):
                tickers_found.append(ticker.abbrev)
        else:
            if (ticker.abbrev in body_set) or (ticker.abbrev in title_set):
                #print("Abbreviation match")
                #print(f"{index}: {ticker.abbrev}")
                if ticker.abbrev not in tickers_found:
                    tickers_found.append(ticker.abbrev)
            elif (ticker.name in body_set) or (ticker.name in title_set):
                #print("Name match")
                #print(f"{index}: {ticker.name}")
                #print(f"{index}: {ticker.abbrev}")
                if ticker.abbrev not in tickers_found:
                    tickers_found.append(ticker.abbrev)
    submissions_df.at[index, "Tickers"]= tickers_found
    print(f"{index}: {tickers_found}")

0: ['MDP']
1: []
2: ['AAPL', 'AES', 'AM', 'AMZN', 'BLL', 'CEL', 'CRWD', 'DG', 'DPZ', 'ET', 'F', 'FB', 'FDX', 'FOR', 'G', 'GM', 'GPN', 'HQY', 'IVZ', 'LPL', 'MCK', 'MKTX', 'MTD', 'NDAQ', 'NEXT', 'NKE', 'OPEN', 'PM', 'ROST', 'S', 'TREE', 'TSLA', 'TWOA', 'U', 'V', 'VFF', 'VUZI', 'W', 'QQQ']
3: ['AMD', 'DD', 'INTC', 'NVDA', 'PE', 'QCOM', 'TSM', 'TWOA', 'UMC']
4: ['CODX']
5: ['TSLA']
6: ['HMNY', 'WISH', 'ARKK']
7: ['TSLA']
8: ['DD', 'RKT']
9: ['AONE', 'B']
10: ['ATH']
11: []
12: []
13: []
14: ['CRM', 'E', 'TA', 'QQQ']
15: []
16: ['CCL', 'CUK']
17: []
18: ['GME', 'HOME', 'OR', 'WWW']
19: ['AAPL', 'AMZN', 'APHA', 'CHPT', 'DIS', 'ARKF', 'PRNT']
20: []
21: ['B', 'DD', 'SC', 'TRIT']
22: ['ABBV', 'ARAV', 'ARDX', 'ATXI', 'AVB', 'AVEO', 'BBIO', 'BIOT', 'BLUE', 'BMY', 'COR', 'FBIO', 'KN', 'KNSA', 'LPTX', 'MRK', 'NWS', 'NWSA', 'PCRX', 'PFE', 'PLX', 'RTX', 'RUBY', 'RYTM', 'SBBP', 'SLDB', 'SLGL', 'SPN', 'SPRB', 'SRPT', 'SUPN', 'UTHR', 'ZEAL']
23: ['CEO', 'DD', 'LMND', 'CFO']
24: ['TWOA', 'ITM']
25: ['AO

In [6]:
submissions_df.head()

,Post_Type,Submission_ID,Title,Author,Body,Distinguished,Num_Comments,Post_ID,URL,Score,Upvote_Ratio,Created_Date_UTC,Body_Words,Title_Words,Tickers
0,Submission,m4inen,How do I cash in a paper stock certificate?,NaN,I'm just now going through my grandparent's de...,NaN,1117,t3_m4inen,https://www.reddit.com/r/stocks/comments/m4ine...,10113,0.95,[1615679015.0],"[Im, just, now, going, through, my, grandparen...","[How, do, I, cash, in, a, paper, stock, certif...",[MDP]
1,Submission,m4zppr,I think some HS friends are running a stocks s...,ElonMuskPaddleBoard,So there’s a bunch of guys I went to high scho...,NaN,639,t3_m4zppr,https://www.reddit.com/r/stocks/comments/m4zpp...,2299,0.92,[1615743393.0],"[So, there, s, a, bunch, of, guys, I, went, to...","[I, think, some, HS, friends, are, running, a,...",[]
2,Submission,m4uv8i,Wall Street Week Ahead for the trading week be...,bigbear0083,Good Sunday morning to all of you here on r/st...,NaN,110,t3_m4uv8i,https://www.reddit.com/r/stocks/comments/m4uv8...,457,0.98,[1615726584.0],"[Good, Sunday, morning, to, all, of, you, here...","[Wall, Street, Week, Ahead, for, the, trading,...","[AAPL, AES, AM, AMZN, BLL, CEL, CRWD, DG, DPZ,..."
3,Submission,m4zabk,My Take on Semiconductors - TSMC DD,datamonkeyz,Reports of the [semiconductor shortage](https:...,NaN,88,t3_m4zabk,https://www.reddit.com/r/stocks/comments/m4zab...,157,0.94,[1615742166.0],"[Reports, of, the, semiconductor, shortage, ht...","[My, Take, on, Semiconductors, TSMC, DD]","[AMD, DD, INTC, NVDA, PE, QCOM, TSM, TWOA, UMC]"
4,Submission,m50u8p,$CODX moving ⬆️!,Scared_Patience_4802,$CODX is still a hidden gem! Totally undervalu...,NaN,29,t3_m50u8p,https://www.reddit.com/r/stocks/comments/m50u8...,133,0.71,[1615746543.0],"[, CODX, is, still, a, hidden, gem, Totally, u...","[, CODX, moving, ]",[CODX]


In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/madisonleopold/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
submissions_df['tokenized_body'] = submissions_df.apply\
(lambda row: nltk.word_tokenize(row['Body']), axis=1)

In [9]:
submissions_df['tokenized_body'].head()

0    [I, 'm, just, now, going, through, my, grandpa...
1    [So, there, ’, s, a, bunch, of, guys, I, went,...
2    [Good, Sunday, morning, to, all, of, you, here...
3    [Reports, of, the, [, semiconductor, shortage,...
4    [$, CODX, is, still, a, hidden, gem, !, Totall...
Name: tokenized_body, dtype: object

In [10]:
for ticker in tickers:
    for index, row in submissions_df.iterrows():
        if ticker.abbrev in submissions_df['tokenized_body']:
            submissions_df['ticker'] = ticker.abbrev

IndentationError: expected an indented block (<ipython-input-10-5268e1975bd8>, line 4)

In [11]:
submissions_df.head()

,Post_Type,Submission_ID,Title,Author,Body,Distinguished,Num_Comments,Post_ID,URL,Score,Upvote_Ratio,Created_Date_UTC,Body_Words,Title_Words,Tickers,tokenized_body
0,Submission,m4inen,How do I cash in a paper stock certificate?,NaN,I'm just now going through my grandparent's de...,NaN,1117,t3_m4inen,https://www.reddit.com/r/stocks/comments/m4ine...,10113,0.95,[1615679015.0],"[Im, just, now, going, through, my, grandparen...","[How, do, I, cash, in, a, paper, stock, certif...",[MDP],"[I, 'm, just, now, going, through, my, grandpa..."
1,Submission,m4zppr,I think some HS friends are running a stocks s...,ElonMuskPaddleBoard,So there’s a bunch of guys I went to high scho...,NaN,639,t3_m4zppr,https://www.reddit.com/r/stocks/comments/m4zpp...,2299,0.92,[1615743393.0],"[So, there, s, a, bunch, of, guys, I, went, to...","[I, think, some, HS, friends, are, running, a,...",[],"[So, there, ’, s, a, bunch, of, guys, I, went,..."
2,Submission,m4uv8i,Wall Street Week Ahead for the trading week be...,bigbear0083,Good Sunday morning to all of you here on r/st...,NaN,110,t3_m4uv8i,https://www.reddit.com/r/stocks/comments/m4uv8...,457,0.98,[1615726584.0],"[Good, Sunday, morning, to, all, of, you, here...","[Wall, Street, Week, Ahead, for, the, trading,...","[AAPL, AES, AM, AMZN, BLL, CEL, CRWD, DG, DPZ,...","[Good, Sunday, morning, to, all, of, you, here..."
3,Submission,m4zabk,My Take on Semiconductors - TSMC DD,datamonkeyz,Reports of the [semiconductor shortage](https:...,NaN,88,t3_m4zabk,https://www.reddit.com/r/stocks/comments/m4zab...,157,0.94,[1615742166.0],"[Reports, of, the, semiconductor, shortage, ht...","[My, Take, on, Semiconductors, TSMC, DD]","[AMD, DD, INTC, NVDA, PE, QCOM, TSM, TWOA, UMC]","[Reports, of, the, [, semiconductor, shortage,..."
4,Submission,m50u8p,$CODX moving ⬆️!,Scared_Patience_4802,$CODX is still a hidden gem! Totally undervalu...,NaN,29,t3_m50u8p,https://www.reddit.com/r/stocks/comments/m50u8...,133,0.71,[1615746543.0],"[, CODX, is, still, a, hidden, gem, Totally, u...","[, CODX, moving, ]",[CODX],"[$, CODX, is, still, a, hidden, gem, !, Totall..."


In [ ]:
# Bag of Words Vectorization
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

# fit and transform the model for submissions body

submissions_body = submissions_df["Body"]

submissions_body_vectors = vectorizer.fit_transform(submissions_body)

#print(submissions_body_vectors.toarray())
#print(submissions_body[0])
#print(submissions_body_vectors[0])

submissions_body_vectors[0].get_feature_names()

In [ ]:
# Using Vader Sentiment Analysis on Submissions
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from pprint import pprint
#nltk.download('vader_lexicon')
sia = SIA()
results = []
submissions_loop = submissions_df[['Body', 'Ticker']]
for index, row in submissions_loop.iterrows():
    body = row[0]
    ticker = row[1]
    pol_score = sia.polarity_scores(body)
    pol_score['body_text'] = body
    pol_score['ticker'] = ticker
    results.append(pol_score)
pprint(results[:3], width=100)

In [ ]:
submission_sentiment_df = pd.DataFrame.from_records(results)
submission_sentiment_df['label'] = 0
submission_sentiment_df.loc[submission_sentiment_df['compound'] > 0.2, 'label'] = 1
submission_sentiment_df.loc[submission_sentiment_df['compound'] < -0.2, 'label'] = -1
submission_sentiment_df.head()

In [ ]:
submission_sentiment_df['body_text'].iloc[0]